In [ ]:
import time
import requests, re
from bs4 import BeautifulSoup

from fake_useragent import UserAgent
ua = UserAgent(verify_ssl=False)

from datetime import datetime, timedelta

from tqdm import tqdm

import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [94]:
headers = {
    'user-agent' : ua.random
}
base_url = 'https://arca.live'

In [257]:
search_frist()

'https://arca.live/b/dunfa?near=2023-05-11T00:00&tz=+0900'

In [256]:
def search_frist():
    
    url = 'https://arca.live/b/dunfa?near=2023-05-11T00:00&tz=+0900'
#     params = {
#         'near' : f'{target_date}T00:00', # 시간은  00:00으로 고정
#         'tz' : '+0900' # +09:00 인듯? timezone인듯?
#     }
    
    return  url


In [211]:
res = get_post_list(base_url, target_date)

In [252]:
soup = BeautifulSoup(res.text, 'html.parser')
df, next_url, is_break = get_post_df(soup, target_date)

In [253]:
next_url

'https://arca.live/b/dunfa?tz=%2B0900&before=226270462'

In [96]:
target_date = '2023-05-11'

In [32]:
res = requests.get(date_url, params=params)

In [33]:
soup = BeautifulSoup(res.text, 'html.parser')

In [34]:
test = soup.select_one('div.list-table.table')

In [61]:
test.select('a.vrow.column')[1]['href']

'/b/dunfa/76052732?near=2023-05-11T00%3A00&tz=%252B0900'

In [63]:
# 게시글 URL
test_url = url + test.select('a.vrow.column')[1]['href']

In [68]:
# 본문
soup2 = BeautifulSoup(res2.text, 'html.parser')
soup2.select_one('meta[name="description"]')['content']

'딜도 카탈로그일듯'

In [193]:
get_next_pages(soup)


'https://arca.live/b/dunfa?tz=%2B0900&before=226270462'

In [192]:
def get_next_pages(soup):
    
    base_url = 'https://arca.live'
    
    before_page = soup.select('div.content-wrapper.clearfix > article > div > nav > ul > li > a')[1]['href']

    return base_url + before_page

In [247]:
def get_post_df(soup, target_date):
    
    target_day = pd.to_datetime(target_date).day
    
    is_break = False
    
    post_lst = soup.select_one('div.list-table.table') # 게시글 리스트
    
    next_url = get_next_pages(soup)
    
    post_urls = []
    post_title = []
    post_writed_at = []
    post_refresh = []
    post_recommand = []
    
#     return post_lst
    
    for cate, info in zip(post_lst.select('span.vcol.col-id'), post_lst.select('a.vrow.column')):
        if cate.text.strip() != '' : continue # 일반 게시글 아니면 패스
        
        title = info.select_one('div.vrow-inner > div.vrow-top > span.vcol.col-title > span.title')
        
        if title:
            title = title.text
            title = re.sub('[\n]+', '', title)
        else:
            title = ''
        
        url = base_url + info['href']
        
        writed_at = info.select_one('div.vrow-inner > div.vrow-bottom > span.vcol.col-time > time')['datetime']
        writed_at = pd.to_datetime(writed_at[:-5]) + timedelta(hours=9)
        
        if writed_at.day != target_day:
            if writed_at.day > target_day:
                is_break = True
            break
        
#         if writed_at:
#             writed_at = writed_at['datetime']
#         else:
#             writed_at = ''
        
        refresh = info.select_one('div.vrow-inner > div.vrow-bottom > span.vcol.col-view').text
        recommand = info.select_one('div.vrow-inner > div.vrow-bottom > span.vcol.col-rate').text
        
        
        post_title.append(title)
        post_urls.append(url)
        post_writed_at.append(writed_at)
        post_refresh.append(refresh)
        post_recommand.append(recommand)
    
    
    df = pd.DataFrame({
        'title': post_title,
        'refresh' : post_refresh,
        'recommand' : post_recommand,
        'url' : post_urls,
        'writed_at': post_writed_at})
    
    
    return df, next_url, is_break

In [ ]:
dfs = []
get_post(dfs, target_date)

In [258]:
def get_post(df_lst, target_date, params = ''):
    res_url = search_frist()
    
    proxy_data = get_proxy_list()
    proxy_idx = 0
    
    while True:
        try:
            proxies = {'http' : 'http://' + proxy_data[proxy_idx][1], 'https' : 'http://' + proxy_data[proxy_idx][1]}
            headers = {
                'user-agent' : ua.random
            }

            res = requests.get(res_url, headers=headers, proxies = proxies)
            print('Requested URL : ', res_url)
            
            soup = BeautifulSoup(res, 'html.parser')
            
            df, res_url, is_break = get_post_df(soup, target_date)
            df_lst.append(df)
            
            if is_break:
                print('break!!')
                break

        
        except Exception as e:
            print(e)
            proxy_idx+=1
            
            if len(proxy_data) <= proxy_idx:
                proxy_idx = 0
                
    return df_lst

In [7]:
def load_data(file_path, load_date = -1):
    y_day = datetime.today() - timedelta(days=1)
    
    if load_date == -1:
    
        load_date = datetime(y_day.year, y_day.month, y_day.day).strftime('%y%m%d')
    
    print(load_date)

    
    path = f'{file_path}_{load_date}.csv'
    
    print(path)

    try:
        df = pd.read_csv(path)
        print('File exist Load FIN!!')
    except:
        df = pd.DataFrame()
        
    return df

In [16]:
def save_daily_data(df, file_path, load_date = -1):
    y_day = datetime.today() - timedelta(days=1)
    
    if load_date == -1:
        load_date = datetime(y_day.year, y_day.month, y_day.day).strftime('%y%m%d')
    
    path = f'{file_path}_{load_date}.csv'
    
    df.to_csv(path, index=False)
    

In [98]:
def get_proxy_list():
    url = 'http://spys.one/en/free-proxy-list/'
    data = {
        'xpp' : '1',
        'xf1' : '0',
        'xf2' : '0',
        'xf4' : '0',
        'xf5' : '2'
    }

    headers = {
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'
    }

    r = requests.post(url, data=data, headers=headers)

    soup = BeautifulSoup(r.content, 'html.parser')

    result = []

    ports = {}
    script = soup.select_one('body > script')
    for row in script.text.split(';'):
        if '^' in row:
            line = row.split('=')
            ports[line[0]]  = line[1].split('^')[0]

    trs = soup.select('tr[onmouseover]')
    for tr in trs:

        is_http = tr.select_one('a')

        if is_http is not None:
            http = is_http.text
#             print(is_http.text)

        e_ip = tr.select_one('font.spy14')
#         print(e_ip)
        ip = ''

        e_port = tr.select_one('script')
        port = ''
        if e_port is not None:
            re_port = re.compile(r'\(([a-zA-Z0-9]+)\^[a-zA-Z0-9]+\)')
            match = re_port.findall(e_port.text)
            for item in match:
                port = port + ports[item]
        else:
            continue

        if e_ip is not None:
            for item in e_ip.findAll('script'):
                item.extract()
            ip = e_ip.text

        else:
            continue

        tds = tr.select("td")
        is_skip = False
        for td in tds:
            e_pct = td.select_one('font > acronym')
            if e_pct is not None:
                pct = re.sub('([0-9]+)%.*', r'\1', e_pct.text)
                if not pct.isdigit():
                    is_skip = True
            else:
                continue

        if not pct.isdigit():
            continue

        result.append((http,ip + ':' + port, pct))

    result.sort(key = lambda element  : int(element[2]), reverse=True)

    return result

In [138]:
# 검색할때 설정해줘야할 것들
path = './data/gimagall'

df = load_data(path)


if not len(df):
    is_concat = False
else:
    is_concat = True

# dc_id = "dnfqq"

if len(df) == 0:
    y_day = datetime.today() - timedelta(days=1)
    y_day_string = str(y_day.year) + "-" + str(y_day.month) + "-" + str(y_day.day)
    last_timestamp = pd.to_datetime(y_day_string)
else:
    last_timestamp = pd.to_datetime(df.sort_values('timestamp', ascending=False)['timestamp'].iloc[0])

print(last_timestamp)

article = article_parse(last_timestamp)

if is_concat:
    print('Concat New Post')
    df = pd.concat([pd.DataFrame(article), df])
else:
    print('Frist Crawling')
    df = pd.DataFrame(article)

save_daily_data(df, path)

230511
./data/gimagall_230511.csv
2023-05-11 00:00:00
Error! :  HTTPSConnectionPool(host='gall.dcinside.com', port=443): Max retries exceeded with url: /mgallery/board/lists?id=dnfqq&page=1&list_num=100 (Caused by ProxyError('Cannot connect to proxy.', ConnectionResetError(54, 'Connection reset by peer')))
Error! :  HTTPSConnectionPool(host='gall.dcinside.com', port=443): Max retries exceeded with url: /mgallery/board/lists?id=dnfqq&page=1&list_num=100 (Caused by ProxyError('Cannot connect to proxy.', ConnectionResetError(54, 'Connection reset by peer')))
Error! :  HTTPSConnectionPool(host='gall.dcinside.com', port=443): Max retries exceeded with url: /mgallery/board/lists?id=dnfqq&page=1&list_num=100 (Caused by ProxyError('Cannot connect to proxy.', ConnectionResetError(54, 'Connection reset by peer')))
Error! :  HTTPSConnectionPool(host='gall.dcinside.com', port=443): Max retries exceeded with url: /mgallery/board/lists?id=dnfqq&page=1&list_num=100 (Caused by ProxyError('Cannot conne

Error! :  HTTPSConnectionPool(host='gall.dcinside.com', port=443): Max retries exceeded with url: /mgallery/board/lists?id=dnfqq&page=19&list_num=100 (Caused by ProxyError('Cannot connect to proxy.', ConnectionResetError(54, 'Connection reset by peer')))
Error! :  HTTPSConnectionPool(host='gall.dcinside.com', port=443): Max retries exceeded with url: /mgallery/board/lists?id=dnfqq&page=19&list_num=100 (Caused by ProxyError('Cannot connect to proxy.', ConnectionResetError(54, 'Connection reset by peer')))
Error! :  HTTPSConnectionPool(host='gall.dcinside.com', port=443): Max retries exceeded with url: /mgallery/board/lists?id=dnfqq&page=19&list_num=100 (Caused by ProxyError('Cannot connect to proxy.', ConnectionResetError(54, 'Connection reset by peer')))
Error! :  HTTPSConnectionPool(host='gall.dcinside.com', port=443): Max retries exceeded with url: /mgallery/board/lists?id=dnfqq&page=19&list_num=100 (Caused by ProxyError('Cannot connect to proxy.', RemoteDisconnected('Remote end clos

Error! :  HTTPSConnectionPool(host='gall.dcinside.com', port=443): Max retries exceeded with url: /mgallery/board/lists?id=dnfqq&page=23&list_num=100 (Caused by ProxyError('Cannot connect to proxy.', ConnectionResetError(54, 'Connection reset by peer')))
Error! :  HTTPSConnectionPool(host='gall.dcinside.com', port=443): Max retries exceeded with url: /mgallery/board/lists?id=dnfqq&page=23&list_num=100 (Caused by ProxyError('Cannot connect to proxy.', RemoteDisconnected('Remote end closed connection without response')))
Error! :  HTTPSConnectionPool(host='gall.dcinside.com', port=443): Max retries exceeded with url: /mgallery/board/lists?id=dnfqq&page=23&list_num=100 (Caused by ProxyError('Cannot connect to proxy.', ConnectionResetError(54, 'Connection reset by peer')))
Error! :  HTTPSConnectionPool(host='gall.dcinside.com', port=443): Read timed out.
Error! :  HTTPSConnectionPool(host='gall.dcinside.com', port=443): Max retries exceeded with url: /mgallery/board/lists?id=dnfqq&page=23&

Error! :  HTTPSConnectionPool(host='gall.dcinside.com', port=443): Max retries exceeded with url: /mgallery/board/lists?id=dnfqq&page=27&list_num=100 (Caused by ProxyError('Cannot connect to proxy.', RemoteDisconnected('Remote end closed connection without response')))
Error! :  HTTPSConnectionPool(host='gall.dcinside.com', port=443): Max retries exceeded with url: /mgallery/board/lists?id=dnfqq&page=27&list_num=100 (Caused by ProxyError('Cannot connect to proxy.', ConnectionResetError(54, 'Connection reset by peer')))
FIND TARGET DATE POST
Data CNT :  1175
custom_break : False -  2023-05-11 20:50:35
Error! :  HTTPSConnectionPool(host='gall.dcinside.com', port=443): Max retries exceeded with url: /mgallery/board/lists?id=dnfqq&page=28&list_num=100 (Caused by ProxyError('Cannot connect to proxy.', ConnectionResetError(54, 'Connection reset by peer')))
Error! :  HTTPSConnectionPool(host='gall.dcinside.com', port=443): Max retries exceeded with url: /mgallery/board/lists?id=dnfqq&page=28&

FIND TARGET DATE POST
Data CNT :  1571
custom_break : False -  2023-05-11 19:56:28
Error! :  HTTPSConnectionPool(host='gall.dcinside.com', port=443): Max retries exceeded with url: /mgallery/board/lists?id=dnfqq&page=32&list_num=100 (Caused by ProxyError('Cannot connect to proxy.', ConnectionResetError(54, 'Connection reset by peer')))
Error! :  HTTPSConnectionPool(host='gall.dcinside.com', port=443): Max retries exceeded with url: /mgallery/board/lists?id=dnfqq&page=32&list_num=100 (Caused by ProxyError('Cannot connect to proxy.', ConnectionResetError(54, 'Connection reset by peer')))
Error! :  HTTPSConnectionPool(host='gall.dcinside.com', port=443): Max retries exceeded with url: /mgallery/board/lists?id=dnfqq&page=32&list_num=100 (Caused by ProxyError('Cannot connect to proxy.', ConnectionResetError(54, 'Connection reset by peer')))
Error! :  HTTPSConnectionPool(host='gall.dcinside.com', port=443): Max retries exceeded with url: /mgallery/board/lists?id=dnfqq&page=32&list_num=100 (C

...

Error! :  HTTPSConnectionPool(host='gall.dcinside.com', port=443): Read timed out.
Error! :  HTTPSConnectionPool(host='gall.dcinside.com', port=443): Max retries exceeded with url: /mgallery/board/lists?id=dnfqq&page=77&list_num=100 (Caused by ProxyError('Cannot connect to proxy.', OSError('Tunnel connection failed: 302 Found')))
Error! :  HTTPSConnectionPool(host='gall.dcinside.com', port=443): Max retries exceeded with url: /mgallery/board/lists?id=dnfqq&page=77&list_num=100 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7fa3c915f5b0>, 'Connection to 185.6.9.159 timed out. (connect timeout=10)'))
Error! :  HTTPSConnectionPool(host='gall.dcinside.com', port=443): Max retries exceeded with url: /mgallery/board/lists?id=dnfqq&page=77&list_num=100 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7fa3c915fd30>, 'Connection to 196.204.24.251 timed out. (connect timeout=10)'))
FIND TARGET DATE POST
Data CNT :  6097
custom_break :

Error! :  HTTPSConnectionPool(host='gall.dcinside.com', port=443): Max retries exceeded with url: /mgallery/board/lists?id=dnfqq&page=81&list_num=100 (Caused by ProxyError('Cannot connect to proxy.', ConnectionResetError(54, 'Connection reset by peer')))
Error! :  HTTPSConnectionPool(host='gall.dcinside.com', port=443): Max retries exceeded with url: /mgallery/board/lists?id=dnfqq&page=81&list_num=100 (Caused by ProxyError('Cannot connect to proxy.', RemoteDisconnected('Remote end closed connection without response')))
Error! :  HTTPSConnectionPool(host='gall.dcinside.com', port=443): Max retries exceeded with url: /mgallery/board/lists?id=dnfqq&page=81&list_num=100 (Caused by ProxyError('Cannot connect to proxy.', ConnectionResetError(54, 'Connection reset by peer')))
Error! :  'NoneType' object is not subscriptable
Error! :  HTTPSConnectionPool(host='gall.dcinside.com', port=443): Max retries exceeded with url: /mgallery/board/lists?id=dnfqq&page=81&list_num=100 (Caused by ProxyError

Error! :  HTTPSConnectionPool(host='gall.dcinside.com', port=443): Max retries exceeded with url: /mgallery/board/lists?id=dnfqq&page=84&list_num=100 (Caused by ProxyError('Cannot connect to proxy.', ConnectionResetError(54, 'Connection reset by peer')))
FIND TARGET DATE POST
Data CNT :  6794
custom_break : False -  2023-05-11 08:13:19
Error! :  HTTPSConnectionPool(host='gall.dcinside.com', port=443): Max retries exceeded with url: /mgallery/board/lists?id=dnfqq&page=85&list_num=100 (Caused by ProxyError('Cannot connect to proxy.', ConnectionResetError(54, 'Connection reset by peer')))
Error! :  HTTPSConnectionPool(host='gall.dcinside.com', port=443): Max retries exceeded with url: /mgallery/board/lists?id=dnfqq&page=85&list_num=100 (Caused by ProxyError('Cannot connect to proxy.', ConnectionResetError(54, 'Connection reset by peer')))
Error! :  HTTPSConnectionPool(host='gall.dcinside.com', port=443): Max retries exceeded with url: /mgallery/board/lists?id=dnfqq&page=85&list_num=100 (C

Error! :  HTTPSConnectionPool(host='gall.dcinside.com', port=443): Read timed out.
Error! :  HTTPSConnectionPool(host='gall.dcinside.com', port=443): Max retries exceeded with url: /mgallery/board/lists?id=dnfqq&page=89&list_num=100 (Caused by ProxyError('Cannot connect to proxy.', OSError('Tunnel connection failed: 302 Found')))
Error! :  HTTPSConnectionPool(host='gall.dcinside.com', port=443): Max retries exceeded with url: /mgallery/board/lists?id=dnfqq&page=89&list_num=100 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7fa3c62f11c0>, 'Connection to 185.6.9.159 timed out. (connect timeout=10)'))
Error! :  HTTPSConnectionPool(host='gall.dcinside.com', port=443): Max retries exceeded with url: /mgallery/board/lists?id=dnfqq&page=89&list_num=100 (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out')))
FIND TARGET DATE POST
Data CNT :  7290
custom_break : False -  2023-05-11 03:42:08
Error! :  HTTPSConnectionPool(host='gall.dcinside.com', 

Error! :  HTTPSConnectionPool(host='gall.dcinside.com', port=443): Max retries exceeded with url: /mgallery/board/lists?id=dnfqq&page=93&list_num=100 (Caused by ProxyError('Cannot connect to proxy.', RemoteDisconnected('Remote end closed connection without response')))
Error! :  HTTPSConnectionPool(host='gall.dcinside.com', port=443): Max retries exceeded with url: /mgallery/board/lists?id=dnfqq&page=93&list_num=100 (Caused by ProxyError('Cannot connect to proxy.', ConnectionResetError(54, 'Connection reset by peer')))
FIND TARGET DATE POST
Data CNT :  7684
custom_break : False -  2023-05-11 01:31:01
Error! :  HTTPSConnectionPool(host='gall.dcinside.com', port=443): Max retries exceeded with url: /mgallery/board/lists?id=dnfqq&page=94&list_num=100 (Caused by ProxyError('Cannot connect to proxy.', ConnectionResetError(54, 'Connection reset by peer')))
Error! :  HTTPSConnectionPool(host='gall.dcinside.com', port=443): Max retries exceeded with url: /mgallery/board/lists?id=dnfqq&page=94&

In [10]:
# 불러오기
path = './data/gimagall'
load_date = '230511'
df = load_data(path, load_date)

230511
./data/gimagall_230511.csv
File exist Load FIN!!


In [15]:
df['link'] = df['link'].str.replace('.com//', '.com/')

In [26]:
df[df['content'] == '']

,num,title,content,reply,refresh,recommand,timestamp,link
137,6333509,서버 악깡버로 안닫네 진짜,,NaN,41,0,2023-05-11 23:39:14,https://gall.dcinside.com/mgallery/board/view/...
697,6332904,모로스 초월 피통 404억 맞음?,,[4],110,0,2023-05-11 22:06:34,https://gall.dcinside.com/mgallery/board/view/...
851,6332738,아까 회랑에서 존나 억울한일 있었음,,NaN,99,0,2023-05-11 21:38:07,https://gall.dcinside.com/mgallery/board/view/...
1097,6332478,4유효 딥다귀 떴냐?,,[10],197,2,2023-05-11 21:02:28,https://gall.dcinside.com/mgallery/board/view/...
1112,6332461,나만 레기온 던전 이상한거냐?,,NaN,41,0,2023-05-11 21:00:55,https://gall.dcinside.com/mgallery/board/view/...
...,...,...,...,...,...,...,...,...
8002,6325194,블레랑 검제 자강두천 아니냐,,[1],88,0,2023-05-11 00:00:58,https://gall.dcinside.com/mgallery/board/view/...
8003,6325193,휘장팔까말까 존나 고민됨,,NaN,49,0,2023-05-11 00:00:42,https://gall.dcinside.com/mgallery/board/view/...
8004,6325192,내일 바로 레어휘장 vs 커먼+젬커먼4개,,NaN,54,0,2023-05-11 00:00:32,https://gall.dcinside.com/mgallery/board/view/...
8005,6325191,이거 형상 팔면 사냐,,[1],126,0,2023-05-11 00:00:29,https://gall.dcinside.com/mgallery/board/view/...


In [139]:
df = df.sort_values('timestamp').reset_index()

In [143]:
df = df[df['timestamp'] < '2023-05-12 00:00:00']

In [ ]:
proxy_data = get_proxy_list()
proxy_idx = 0 

# NaN -> ''로 변경 
# df['content'].loc[df[df['content'].isna()].index] = ''

for idx, link in tqdm(enumerate(df['link'].values)):
    if len(df['content'].iloc[idx]) :
        continue
    print('SEARCHING Iloc : ', idx)
    df['content'].iloc[idx], proxy_idx = get_post_content(link, proxy_data, proxy_idx)
    
save_daily_data(df, path)

0it [00:00, ?it/s]

SEARCHING Iloc :  137
0


138it [00:03, 39.48it/s]

Find Content!
SEARCHING Iloc :  697
0


698it [00:06, 119.01it/s]

...

Find Content!
SEARCHING Iloc :  1917
0
Find Content!


1918it [54:58,  3.69s/it]

SEARCHING Iloc :  1918
0


1919it [55:01,  3.66s/it]

Find Content!
SEARCHING Iloc :  1919
0


In [17]:
save_daily_data(df, path, load_date)

In [20]:
df['content'].iloc[95]

'\n본캐 하나만 3단가고 나머지는 2개는\xa02단해야될거같은데\n'

In [96]:
proxies = {'http' : 'http://' + proxy_data[proxy_idx][1], 'https' : 'http://' + proxy_data[proxy_idx][1]}
headers = {
        'user-agent' : ua.random
    }
res = requests.get(link, headers=headers, proxies = proxies, timeout=(10,10))